# Object Detection
This notebook will serve as the primary means of documentation for creating an object detection model
_______________________________________________________________________________________________________________________________________________

### Full group includes:
- Jordan Brown
- Dylan Roy
- Maxwell Barret
- Julia Dewhurt

### Primary notebook contributers:
- ""
- ""
_______________________________________________________________________________________________________________________________________________

***The primary notebook contributers are the group members who were assigned to this specific task. All group members will work in collaboration to create a final working product. However, the nature of this project calls for the full group to be assigned to primary tasks.***

## Notebook Goal/Purpose

This notebook will be used to create a model for traffic based object detection. This will involve using labeled image and video feeds to determine what traffic objects look like. This model will be able to detect and track traffic counts at individual intersections. This tracking will be used to create a dataset which can hopefully be used to train a traffic prediction for the traffic analysis portion of this project.
_______________________________________________________________________________________________________________________________________________

In [1]:
import os
import zipfile
import requests

def download_file(url, destination):
    """
    Download a file from a URL to a specified destination.
    """
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Downloaded {destination}")
    else:
        print(f"Failed to download {url}")

def extract_zip(zip_file, extract_to):
    """
    Extract a zip file to a specified directory.
    """
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_file} to {extract_to}")

def download_and_extract_coco(destination_dir):
    """
    Download and extract the COCO dataset (2017 version) images and annotations.
    """
    # Create destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Define COCO dataset URLs
    coco_urls = {
        'train2017': 'http://images.cocodataset.org/zips/train2017.zip',
        'val2017': 'http://images.cocodataset.org/zips/val2017.zip',
        'annotations': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip',
    }

    # Download images and annotations
    for name, url in coco_urls.items():
        file_path = os.path.join(destination_dir, f"{name}.zip")
        
        print(f"Downloading {name}...")
        download_file(url, file_path)
        
        # Extract the downloaded zip files
        print(f"Extracting {name}...")
        extract_zip(file_path, destination_dir)

        # Remove zip files after extraction
        os.remove(file_path)

    print("COCO dataset downloaded and extracted successfully!")

# Check if the COCO dataset has already been downloaded
if os.path.exists('COCO'):
    print("COCO dataset already exists. Skipping download and extraction.")
    exit()
    
# Specify the directory to save the dataset
destination_dir = 'COCO'

# Start downloading and extracting COCO dataset
download_and_extract_coco(destination_dir)


Downloaded COCO/train2017.zip
Extracting train2017...
Extracted COCO/train2017.zip to COCO
Downloaded COCO/val2017.zip
Extracting val2017...
Extracted COCO/val2017.zip to COCO
Downloaded COCO/annotations.zip
Extracting annotations...
Extracted COCO/annotations.zip to COCO
COCO dataset downloaded and extracted successfully!


In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models.detection import faster_rcnn
from pycocotools.coco import COCO
import os

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# COCO Dataset Paths (adjust according to where you downloaded COCO dataset)
train_img_dir = 'path/to/coco/train2017'
val_img_dir = 'path/to/coco/val2017'
train_annot_file = 'path/to/coco/annotations/instances_train2017.json'
val_annot_file = 'path/to/coco/annotations/instances_val2017.json'

# Load COCO datasets using torchvision's CocoDetection
train_dataset = torchvision.datasets.CocoDetection(root=train_img_dir, annFile=train_annot_file, transform=transform)
val_dataset = torchvision.datasets.CocoDetection(root=val_img_dir, annFile=val_annot_file, transform=transform)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Load pre-trained Faster R-CNN model with a ResNet-50 backbone
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Modify the model to detect vehicles (2 classes: background and vehicle)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)  # 2 classes: background and vehicle

# Use GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Define learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Forward pass
        loss_dict = model(images, targets)
        
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass and optimize
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    
    lr_scheduler.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item()}")

    # Validation (optional, to track the progress)
    if (epoch + 1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            for images, targets in val_loader:
                images = [image.to(device) for image in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                predictions = model(images)
                # Here, you can visualize the predictions or compute metrics like mAP

# Save the trained model
torch.save(model.state_dict(), 'vehicle_detection_model.pth')

print("Training complete!")
